# Single Replica TIS

This notebook shows how to run single replica TIS move scheme. This assumes you can load engine, network, and initial sample from a previous calculation.

In [ ]:
%matplotlib inline
import openpathsampling as paths
import numpy as np

In [ ]:
from openpathsampling.visualize import PathTreeBuilder, PathTreeBuilder
from IPython.display import SVG, HTML

def ipynb_visualize(movevis):
    """Default settings to show a movevis in an ipynb."""
    view = movevis.renderer
    view.zoom = 1.5
    view.scale_y = 18
    view.scale_th = 20
    view.font_size = 0.4
    return view

Open the storage and load things from it.

In [ ]:
old_store = paths.AnalysisStorage("mstis.nc")

In [ ]:
network = old_store.networks[0]
engine = old_store.engines[0]
template = old_store.snapshots[0]

In [ ]:
network.hist_args['max_lambda'] = { 'bin_width' : 0.02, 'bin_range' : (0.0, 0.5) }
network.hist_args['pathlength'] = { 'bin_width' : 5, 'bin_range' : (0, 150) }
rate = network.rate_matrix(old_store)

In [ ]:
# TODO: add way to get the 
table = {}
for t in network.sampling_transitions:
    for ens, lambda_ens in zip(t.ensembles, [0.04, 0.09, 0.15]):
        table[ens] = t.tcp(lambda_ens)
for ens in network.special_ensembles['ms_outer'].keys():
    table[ens] = t.tcp(0.25)

In [ ]:
bias = paths.BiasEnsembleTable(table)

In [ ]:
print table

Here we actually set up the SRTIS move scheme for the given network. It only requires one line:

In [ ]:
scheme = paths.SRTISScheme(network, bias)

Now we'll visualize the SRTIS move scheme.

In [ ]:
movevis = paths.visualize.MoveTreeBuilder()
movevis.mover(scheme.move_decision_tree(), network.all_ensembles)
SVG(ipynb_visualize(movevis).to_svg())

Next we need to set up an appropriate single-replica initial sample set. We'll take the last version of from one of the outer TIS ensembles.

In [ ]:
final_samp0 = old_store.steps[len(old_store.steps)-1].active[network.sampling_ensembles[-1]]

In [ ]:
sset = paths.SampleSet([final_samp0])

Finally, we set up the new storage file and the new simulation. 

In [ ]:
storage = paths.Storage("srtis.nc", "w", template)

In [ ]:
srtis = paths.PathSampling(
    storage=storage,
    globalstate=sset,
    engine=engine,
    move_scheme=scheme
)

In [ ]:
n_steps_to_run = int(scheme.n_steps_for_trials(
        mover=scheme.movers['minus'][0], 
        n_attempts=2
    ))
print n_steps_to_run

In [ ]:
# logging creates ops_output.log file with details of what the calculation is doing
import logging.config
logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

In [ ]:
srtis.run_until(n_steps_to_run)

In [ ]:
storage.close()
storage = paths.AnalysisStorage("srtis.nc")

In [ ]:
scheme = storage.schemes[0]

In [ ]:
scheme.move_summary(storage)

In [ ]:
scheme.move_summary(storage, 'shooting')

In [ ]:
scheme.move_summary(storage, 'minus')

In [ ]:
scheme.move_summary(storage, 'repex')